# Gauss-Newton method

In [ ]:
%matplotlib inline

In [ ]:
import copy
import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import random
import re

from IPython.display import display, Math
from scipy import optimize
from scipy import linalg as la
from scipy import special

from functools import reduce

In [ ]:
def is_digit(x):
    if re.match("^-?\d+?\.\d*?$", x) is None:
        return False
    return True

In [ ]:
def gauss_newton(F, dF, init_vec, tol=5e-6, max_iter=100, displayQ=False):
    convergedQ = False
    curr_vec = copy.deepcopy(init_vec)
    for j in range(max_iter):
        f = np.array(F(*curr_vec))
        jacobian = np.array(dF(*curr_vec))
        
        q, r = np.linalg.qr(jacobian)
        p = -np.dot(q.T, f)
        v = la.solve_triangular(r, p)
        ls = optimize.nonlin._nonlin_line_search(lambda u: F(*u), curr_vec, f, v)
        v *= ls[0]
        
        curr_vec += v
        
        if displayQ:
            print('iter: {}, x = {}, step = {}, |d| = {}'.format(j, curr_vec, ls[0], np.linalg.norm(v, np.inf)))
        if np.linalg.norm(v, np.inf) < tol*np.linalg.norm(curr_vec, np.inf):
            if displayQ:
                print('converged: iter = {}, x = {}, |d| = {}'.format(j, curr_vec, np.linalg.norm(v, np.inf)))
            convergedQ = True
            break
    
    return curr_vec, convergedQ

## fitting a circle

In [ ]:
radius_true = 3.14
x0_true = 1.1
y0_true = -0.7

In [ ]:
def rI(x, y, x0, y0, R):
    return (x - x0)**2 + (y - y0)**2 -R**2

def rAcu(x, y, x0, y0, R):
    assert len(x) == len(y), 'Error: lengths of x ({}) and y ({}) are different'.format(len(x), len(y))
    return np.array([rI(i, j, x0, y0, R) for i,j in zip(x, y)])

def drAcu(x, y, x0, y0, R):
    assert len(x) == len(y), 'Error: lengths of x ({}) and y ({}) are different'.format(len(x), len(y))
    return -2*np.array([[i - x0, j - y0, R] for i,j in zip(x, y)])

In [ ]:
error_amplitude = radius_true*0.1
number_of_points = 101

x_data = []
y_data = []
theta_data = np.linspace(0, 2*math.pi, number_of_points)
for theta in theta_data:
    x_data.append(x0_true + radius_true*math.cos(theta) + error_amplitude*(1 - 2*random.random()))
    y_data.append(y0_true + radius_true*math.sin(theta) + error_amplitude*(1 - 2*random.random()))

plt.figure(figsize=(10,10))
plt.style.use('dark_background')
plt.rcParams.update({'font.size': 14})
plt.rc('font')
plt.xlabel(r'x', fontsize=16)
plt.ylabel(r'y', fontsize=16)

plt.plot(x_data, y_data, 'og')

plt.show()

In [ ]:
opt, is_converged = gauss_newton(lambda x0, y0, R: rAcu(x_data, y_data, x0, y0, R),
                                 lambda x0, y0, R: drAcu(x_data, y_data, x0, y0, R), [0,0,1], tol=1e-6, displayQ=True)

if is_converged:
    print('\nTrue radius: {0:0.4f}, fitted radius: {1:0.4f}'.format(radius_true, opt[2]))
    print('True center: ({0:0.4f},{1:0.4f}), fitted center: ({2:0.4f},{3:0.4f})'.format(x0_true, y0_true, opt[0], opt[1]))
else:
    print('Gauss-Newton method did not converge.')

In [ ]:
for theta in theta_data:
    x_data.append(x0_true + radius_true*math.cos(theta) + error_amplitude*random.random())
    y_data.append(y0_true + radius_true*math.sin(theta) + error_amplitude*random.random())

x_true = [x0_true + radius_true*math.cos(theta) for theta in theta_data]
y_true = [y0_true + radius_true*math.sin(theta) for theta in theta_data]

x_pred = [opt[0] + opt[2]*math.cos(theta) for theta in theta_data]
y_pred = [opt[1] + opt[2]*math.sin(theta) for theta in theta_data]

plt.figure(figsize=(10, 10))
plt.rc('font')
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.gca().set_aspect('equal', 'datalim')

plt.plot(x_pred, y_pred,'-r')
plt.plot(x_true, y_true,'--y')
plt.plot(x_data, y_data, 'og')

plt.show()

## Colebrook correlation
$$\frac{1}{\sqrt{f}} = -2\log_{10}\left(\frac{e/D}{3.7}+\frac{2.51}{\mathrm{Rey}\sqrt{f}}\right)$$

In [ ]:
raw = []
with open('pipe.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        if is_digit(row[0]):
            raw.append([float(x) for x in row])
data = np.array(raw)
x_data = data[:,0]
y_data = data[:,1]
plt.figure(figsize=(10,10))
plt.style.use('dark_background')
plt.plot(x_data, y_data, 'o')
plt.xlabel('Rey')
plt.ylabel(r'$\frac{8\tau}{\rho U^2}$')
plt.show()

In [ ]:
def rCole(Re, f, x0, x1):
    return 1./math.sqrt(f) + 2*math.log10(x0 + x1/Re/math.sqrt(f))

def rColeAcu(Re, f, x0, x1):
    assert len(Re) == len(f), 'Error: lengths of Re ({}) and f ({}) are different'.format(len(Re), len(f))
    return np.array([rCole(i, j, x0, x1) for i,j in zip(Re, f)])

def drColeAcu(Re, f, x0, x1):
    assert len(Re) == len(f), 'Error: lengths of Re ({}) and f ({}) are different'.format(len(Re), len(f))
    return 2.0/math.log(10.0)*np.array([[1./(x0 + x1/i/math.sqrt(j)), 
                                       1./i/math.sqrt(j)/(x0 + x1/i/math.sqrt(j))] for i,j in zip(Re, f)])

In [ ]:
a_cole = 3.7
b_cole = 2.51
x0_cole = 1./2/69.6/a_cole
x1_cole = 2*b_cole
opt, is_converged = gauss_newton(lambda x0, x1: rColeAcu(x_data, y_data, x0, x1),
                                 lambda x0, x1: drColeAcu(x_data, y_data, x0, x1), [0.001,2.0], tol=1e-6, displayQ=True)

if is_converged:
    display(Math('\n\\text{{Colebrook constants: }}a = {0:0.1f}, b = {1:0.2f}'.format(a_cole, b_cole)))
    display(Math('\n\\text{{Computed constants: }}a = {0:0.4f}, b = {1:0.4f}'.format(1./2/69.6/opt[0], opt[1]/2)))
else:
    print('Gauss-Newton method did not converge.')

In [ ]:
def f_darcy(Re, x0, x1):
    a = x1/Re
    l10 = math.log(10.0)
    denom = np.real(2/l10*special.lambertw(l10/2/a*10**(x0/2/a)) - x0/a)
    return 1/denom**2

In [ ]:
Re_ = np.linspace(x_data[0], x_data[-1], 101)

f_pred = [f_darcy(Re, *opt) for Re in Re_]
f_Cole = [f_darcy(Re, x0_cole, x1_cole) for Re in Re_] 

plt.figure(figsize=(10, 10))
plt.rc('font')
plt.xlabel('Rey', fontsize=16)
plt.ylabel(r'$\frac{8\tau}{\rho U^2}$')

plt.plot(Re_, f_pred,'-r', label='nonlinear fit')
plt.plot(Re_, f_Cole,'-y', label='Colebrook')
plt.plot(x_data, y_data, 'o', label='experiments')
plt.legend()

plt.show()